## 02 - Discover Sentinel-1 pairs for pre-seismic and coseismic coherence change analysis

* Import the Python packages

In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
import os
import sys
import glob

import cioppy
ciop = cioppy.Cioppy()

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import gc

from shapely.geometry import box
from shapely.wkt import loads  

import py_earthquakes

from datetime import datetime, timedelta
import dateutil.parser

import geopandas as gp

import folium

* Use an earthquake

In [27]:
bounding_box = [20, 39, 22, 41]

bbox = box(bounding_box[0],
           bounding_box[1],
           bounding_box[2],
           bounding_box[3])

min_mag = 4

start_date = '2016-10-01'

stop_date = '2016-11-30'

In [18]:
eq_search = py_earthquakes.EarthQuakes(start_date,
                                       stop_date,
                                       min_mag=min_mag,
                                       bbox=bounding_box)

In [47]:
eq_index = 5

In [48]:
print eq_search.earthquakes[eq_index].title
print eq_search.earthquakes[eq_index].id
print eq_search.earthquakes[eq_index].date
print eq_search.earthquakes[eq_index].wkt

M 4.5 - 9km NW of Rodotopion, Greece
us20007els
2016-10-16T18:44:04.530000Z
POINT(20.6459 39.7731)


Create a buffer of 0.5 degrees around the selected earthquake event 

In [23]:
eq_search.earthquakes[eq_index].wkt

'POINT(20.6459 39.7731)'

In [24]:
aoi_wkt = box(*loads(eq_search.earthquakes[eq_index].wkt).buffer(0.5).bounds).wkt

In [25]:
aoi_wkt

'POLYGON ((21.1459 39.2731, 21.1459 40.2731, 20.1459 40.2731, 20.1459 39.2731, 21.1459 39.2731))'

In [30]:
lat = (bounding_box[1] + bounding_box[3])/2
lon = (bounding_box[0] + bounding_box[2])/2

zoom_start = 7

m = folium.Map(location=[lat, lon], zoom_start=zoom_start)

folium.PolyLine(
    locations=np.asarray([t[::-1] for t in list(loads(bbox.wkt).exterior.coords)]).tolist(),
    color='black',
    weight=2,
    tooltip='Area of interest for the earthquake discovery',
).add_to(m)

radius = 4
folium.CircleMarker(
    location=[loads(eq_search.earthquakes[eq_index].wkt).y,
              loads(eq_search.earthquakes[eq_index].wkt).x],
    radius=radius,
    color='#FF0000',
    stroke=False,
    fill=True,
    fill_opacity=1,
    opacity=1,
    popup='{} pixels'.format(radius),
    tooltip=eq_search.earthquakes[0].title,
).add_to(m)

folium.PolyLine(
    locations=np.asarray([t[::-1] for t in list(loads(aoi_wkt).exterior.coords)]).tolist(),
    color='red',
    weight=2,
    tooltip='Area of interest for the earthquake discovery',
).add_to(m)

m.save(os.path.join('maps', 'eq_search.html'))

m

* Search parameters

Set the catalogue endpoint to Sentinel-1:

In [35]:
series = 'https://catalog.terradue.com/sentinel1/search'

Define the end of the time of interest and look for a slave between the event date and six days after:

In [31]:
slave_search_stop_date = (dateutil.parser.parse(eq_search.earthquakes[eq_index].date) + timedelta(days=6)).isoformat()

* Build and submit the catalog search


In [36]:
search_params = dict([('geom', aoi_wkt),
                      ('start', eq_search.earthquakes[eq_index].date),
                      ('stop', slave_search_stop_date),
                      ('pt', 'SLC')])

In [37]:
slave_search = ciop.search(end_point=series,
                           params=search_params,
                           output_fields='self,productType,track,enclosure,identifier,wkt,startdate', 
                           model='EOP')

* Put all slaves in a geodataframe and plot the Sentinel-1 slave candidates

In [38]:
aoi = loads(aoi_wkt)

In [39]:
result = []

locations = []

for index, elem in enumerate(slave_search):
    
    locations.append([t[::-1] for t in list(loads(elem['wkt']).exterior.coords)])
    
    slave_wkt = loads(elem['wkt'])
    
    result.append({'self' : elem['self'],
                   'identifier' : elem['identifier'],
                   'enclosure' : elem['enclosure'],
                   'date' : elem['startdate'],
                   'wkt': loads(elem['wkt']),
                   'aoi_intersec' : (slave_wkt.intersection(aoi).area/aoi.area) * 100,
                   'contains': slave_wkt.contains(aoi)
                  })
    
slaves = gp.GeoDataFrame(result)

In [40]:
lat = loads(eq_search.earthquakes[eq_index].wkt).y
lon = loads(eq_search.earthquakes[eq_index].wkt).x

zoom_start = 7

m = folium.Map(location=[lat, lon], zoom_start=zoom_start)

radius = 4
folium.CircleMarker(
    location=[lat, lon],
    radius=radius,
    color='#FF0000',
    stroke=False,
    fill=True,
    fill_opacity=0.6,
    opacity=1,
    popup='{} pixels'.format(radius),
    tooltip='Earthquake event',
).add_to(m)


folium.PolyLine(
    locations=np.asarray([t[::-1] for t in list(loads(aoi_wkt).exterior.coords)]).tolist(),
    color='#FF0000',
    weight=2,
    tooltip='Sentinel-1 area of interest for discovery',
).add_to(m)

folium.PolyLine(
    locations=locations,
    color='orange',
    weight=1,
    opacity=1,
    smooth_factor=0,
).add_to(m)

m.save(os.path.join('maps', '%s_search.html' % eq_search.earthquakes[eq_index].id))

m

In [41]:
slaves

,aoi_intersec,contains,date,enclosure,identifier,self,wkt
0,93.960487,False,2016-10-18T16:32:06.0438950Z,https://store.terradue.com/download/sentinel1/...,S1A_IW_SLC__1SDV_20161018T163206_20161018T1632...,https://catalog.terradue.com/sentinel1/search?...,"POLYGON ((19.368031 40.835575, 22.396507 41.23..."
1,19.306843,False,2016-10-18T16:31:41.2230520Z,https://store.terradue.com/download/sentinel1/...,S1A_IW_SLC__1SDV_20161018T163141_20161018T1632...,https://catalog.terradue.com/sentinel1/search?...,"POLYGON ((19.736567 39.342892, 22.70108 39.744..."
2,42.871116,False,2016-10-18T04:38:47.9894800Z,https://store.terradue.com/download/sentinel1/...,S1B_IW_SLC__1SDV_20161018T043847_20161018T0439...,https://catalog.terradue.com/sentinel1/search?...,"POLYGON ((22.349596 37.795113, 19.450989 38.19..."
3,70.513769,False,2016-10-18T04:38:22.2086930Z,https://store.terradue.com/download/sentinel1/...,S1B_IW_SLC__1SDV_20161018T043822_20161018T0438...,https://catalog.terradue.com/sentinel1/search?...,"POLYGON ((22.729288 39.285404, 19.763023 39.68..."
4,35.270604,False,2016-10-17T16:39:36.7587880Z,https://store.terradue.com/download/sentinel1/...,S1B_IW_SLC__1SDV_20161017T163936_20161017T1640...,https://catalog.terradue.com/sentinel1/search?...,"POLYGON ((17.319502 40.773991, 20.347933 41.17..."
5,21.041412,False,2016-10-17T16:39:11.9338340Z,https://store.terradue.com/download/sentinel1/...,S1B_IW_SLC__1SDV_20161017T163911_20161017T1639...,https://catalog.terradue.com/sentinel1/search?...,"POLYGON ((17.697649 39.282978, 20.658102 39.68..."
6,27.342190,False,2016-10-17T04:47:39.0485030Z,https://store.terradue.com/download/sentinel1/...,S1A_IW_SLC__1SDV_20161017T044739_20161017T0448...,https://catalog.terradue.com/sentinel1/search?...,"POLYGON ((20.356789 38.021778, 17.475241 38.42..."
7,44.418058,False,2016-10-17T04:47:14.2173840Z,https://store.terradue.com/download/sentinel1/...,S1A_IW_SLC__1SDV_20161017T044714_20161017T0447...,https://catalog.terradue.com/sentinel1/search?...,"POLYGON ((20.725029 39.57457, 17.784842 39.971..."


Select the slave that 'best' covers the AOI

In [42]:
slave = slave_search[slaves['aoi_intersec'].idxmax()]

slave

{'enclosure': 'https://store.terradue.com/download/sentinel1/files/v1/S1A_IW_SLC__1SDV_20161018T163206_20161018T163233_013547_015AEB_712A',
 'identifier': 'S1A_IW_SLC__1SDV_20161018T163206_20161018T163233_013547_015AEB_712A',
 'productType': 'SLC',
 'self': 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20161018T163206_20161018T163233_013547_015AEB_712A',
 'startdate': '2016-10-18T16:32:06.0438950Z',
 'track': '175',
 'wkt': 'POLYGON((19.368031 40.835575,22.396507 41.235806,22.733843 39.616779,19.779461 39.21563,19.368031 40.835575))'}

In [43]:
slave['startdate']

'2016-10-18T16:32:06.0438950Z'

In [46]:
lat = loads(eq_search.earthquakes[eq_index].wkt).y
lon = loads(eq_search.earthquakes[eq_index].wkt).x

zoom_start = 7

m = folium.Map(location=[lat, lon], zoom_start=zoom_start)

radius = 4
folium.CircleMarker(
    location=[lat, lon],
    radius=radius,
    color='#FF0000',
    stroke=False,
    fill=True,
    fill_opacity=0.6,
    opacity=1,
    popup='{} pixels'.format(radius),
    tooltip='Earthquake event',
).add_to(m)


folium.PolyLine(
    locations=np.asarray([t[::-1] for t in list(loads(aoi_wkt).exterior.coords)]).tolist(),
    color='#FF0000',
    weight=2,
    tooltip='Sentinel-1 area of interest for discovery',
).add_to(m)

folium.PolyLine(
    locations=np.asarray([t[::-1] for t in list(loads(slave['wkt']).exterior.coords)]).tolist(),
    color='orange',
    weight=1,
    opacity=1,
    tooltip='Discovered slave',
    smooth_factor=0,
).add_to(m)

m.save(os.path.join('maps', '%s_slave_search.html' % eq_search.earthquakes[eq_index].id))

m

**Search for the pre-event masters**

In [49]:
master_search_start_date = (dateutil.parser.parse(slave['startdate']) + timedelta(days=-24)).isoformat()

In [50]:
master_search_stop_date = (dateutil.parser.parse(slave['startdate']) + timedelta(days=-1)).isoformat()

In [51]:
master_search_params = dict([('geom', slave['wkt']),
                             ('track', slave['track']),
                             ('pt',slave['productType']),
                             ('start', master_search_start_date),
                             ('stop', master_search_stop_date)])

In [52]:
try:
    master_search = ciop.search(end_point=series, 
                            params=master_search_params,
                            output_fields='identifier,enclosure,self,startdate,wkt',
                            model='EOP')
except IndexError:
    print('no masters')

In [54]:
result = []

for index, elem in enumerate(master_search):
    
    master_wkt = loads(elem['wkt'])
    
    result.append({'self' : elem['self'],
                   'identifier' : elem['identifier'],
                   'enclosure' : elem['enclosure'],
                   'wkt': loads(elem['wkt']),
                   'aoi_intersec' : (master_wkt.intersection(aoi).area/aoi.area) * 100,
                   'slave_intersec' : (master_wkt.intersection(loads(slave['wkt']))).area / loads(slave['wkt']).area * 100,
                   'contains': master_wkt.contains(aoi),
                   'days': (dateutil.parser.parse(slave['startdate']) - dateutil.parser.parse(elem['startdate'])).days
                  })
    
masters = gp.GeoDataFrame(result)

In [55]:
masters

,aoi_intersec,contains,days,enclosure,identifier,self,slave_intersec,wkt
0,0.000000,False,11,https://store.terradue.com/download/sentinel1/...,S1A_IW_SLC__1SDV_20161006T163230_20161006T1632...,https://catalog.terradue.com/sentinel1/search?...,8.059791,"POLYGON ((18.973026 42.324501, 22.077484 42.72..."
1,94.021193,False,11,https://store.terradue.com/download/sentinel1/...,S1A_IW_SLC__1SDV_20161006T163206_20161006T1632...,https://catalog.terradue.com/sentinel1/search?...,99.934219,"POLYGON ((19.367752 40.834766, 22.396078 41.23..."
2,19.242306,False,12,https://store.terradue.com/download/sentinel1/...,S1A_IW_SLC__1SDV_20161006T163141_20161006T1632...,https://catalog.terradue.com/sentinel1/search?...,7.805433,"POLYGON ((19.736254 39.342201, 22.700668 39.74..."
3,14.438422,False,18,https://store.terradue.com/download/sentinel1/...,S1B_IW_SLC__1SDV_20160930T163136_20160930T1632...,https://catalog.terradue.com/sentinel1/search?...,52.685858,"POLYGON ((19.140676 41.6096, 22.230778 42.0129..."
4,98.112574,False,18,https://store.terradue.com/download/sentinel1/...,S1B_IW_SLC__1SDV_20160930T163111_20160930T1631...,https://catalog.terradue.com/sentinel1/search?...,55.609042,"POLYGON ((19.522146 40.1185, 22.541943 40.5227..."
5,0.000000,False,23,https://store.terradue.com/download/sentinel1/...,S1A_IW_SLC__1SDV_20160924T163230_20160924T1632...,https://catalog.terradue.com/sentinel1/search?...,8.060515,"POLYGON ((18.957047 42.382229, 22.06447 42.782..."
6,94.049465,False,23,https://store.terradue.com/download/sentinel1/...,S1A_IW_SLC__1SDV_20160924T163206_20160924T1632...,https://catalog.terradue.com/sentinel1/search?...,99.916430,"POLYGON ((19.367441 40.834682, 22.395617 41.23..."
7,19.224740,False,24,https://store.terradue.com/download/sentinel1/...,S1A_IW_SLC__1SDV_20160924T163141_20160924T1632...,https://catalog.terradue.com/sentinel1/search?...,7.793974,"POLYGON ((19.735977 39.341984, 22.700241 39.74..."


* Select the two masters according to the ranking of AOI coverage and nearest cycles in time

In [61]:
master_1 = master_search[masters.sort_values(['aoi_intersec', 'days'], ascending=[False, False]).iloc[0].name]
master_2 = master_search[masters.sort_values(['aoi_intersec', 'days'], ascending=[False, False]).iloc[1].name]

In [62]:
s1_identifiers = []
s1_references = [] 
locations = []

for product in [slave, master_1, master_2]:
    
    locations.append([t[::-1] for t in list(loads(product['wkt']).exterior.coords)])
    
    s1_identifiers.append(product['identifier'])
    s1_references.append(product['self'])


Plot the Sentinel-1 products (slave, master 1 and master 2), the earthquake point and its area of interest

In [63]:
lat = loads(eq_search.earthquakes[eq_index].wkt).y
lon = loads(eq_search.earthquakes[eq_index].wkt).x

zoom_start = 7

m = folium.Map(location=[lat, lon], zoom_start=zoom_start)

radius = 4
folium.CircleMarker(
    location=[lat, lon],
    radius=radius,
    color='#FF0000',
    stroke=False,
    fill=True,
    fill_opacity=0.6,
    opacity=1,
    popup='{} pixels'.format(radius),
    tooltip='I am in pixels',
).add_to(m)


folium.PolyLine(
    locations=np.asarray([t[::-1] for t in list(loads(aoi_wkt).exterior.coords)]).tolist(),
    color='#FF0000',
    weight=2,
    tooltip='Japan flooding',
).add_to(m)

folium.PolyLine(
    locations=locations,
    color='orange',
    weight=1,
    opacity=1,
    smooth_factor=0,
).add_to(m)

m.save(os.path.join('maps', '%s_final.html' % eq_search.earthquakes[eq_index].id))

m

In [72]:
print ('s1_references = %s' % s1_references)

s1_references = ['https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20161018T163206_20161018T163233_013547_015AEB_712A', 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1B_IW_SLC__1SDV_20160930T163111_20160930T163139_002301_003E3A_4516', 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20160924T163206_20160924T163233_013197_014FD9_29A5']


In [66]:
pair_coseismic = [s1_references[0], 
                 s1_references[1]]

pair_preseismic = [s1_references[1], 
                 s1_references[2]]


In [70]:
print ('pair_coseismic = %s' % pair_coseismic)

pair_coseismic = ['https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20161018T163206_20161018T163233_013547_015AEB_712A', 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1B_IW_SLC__1SDV_20160930T163111_20160930T163139_002301_003E3A_4516']


In [71]:
print ('pair_preseismic = %s' % pair_preseismic)

pair_preseismic = ['https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1B_IW_SLC__1SDV_20160930T163111_20160930T163139_002301_003E3A_4516', 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20160924T163206_20160924T163233_013197_014FD9_29A5']


## License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.